In [1]:
from utils import *
import numpy as np
import os
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [105]:
def project_vertices(vertices, frame_matrix, image_size):
    # Берём только 3х3 матрицу вращения и 3х1 вектор смещения
    R = frame_matrix[:3, :3]
    T = frame_matrix[:3, 3]
    f = 1.0
    H = np.vstack([R * f, T])

    # Ортографическая проекция
    projected = vertices @ H[:3, :3].T + H[:3, 2]
    projected = projected[:, :2]

    # Масштабируем до размера изображения
    projected -= projected.min(axis=0)
    projected /= projected.max(axis=0)
    projected *= np.array(image_size)[::-1]  # W x H
    return projected


def build_triangle_index_map(mesh, image_size):
    vertices = mesh["vertices"]
    faces = mesh["faces"]
    frame_matrix = mesh["frame_matrix"]
    projected_vertices = project_vertices(vertices, frame_matrix, image_size)
    index_map = np.full(image_size[::-1], -1, dtype=np.int32)  # (H, W)

    for tri_idx, face in enumerate(faces):
        pts = projected_vertices[face].astype(np.int32)
        cv2.fillPoly(index_map, [pts], color=tri_idx)

    return index_map

1. Считываем данные и приводим к единой системе координат

In [106]:
# Размер воксельного пространства
VOXEL_GRID_SIZE = 256
# Размер проекции
image_size = (VOXEL_GRID_SIZE * 2, VOXEL_GRID_SIZE * 2)

In [107]:
# Получим пути к исходным файлам
files = [f for f in os.listdir("data") if f.endswith('.x')]

In [108]:
meshes = []
for f in files:
    print(f)
    filepath = os.path.abspath(f'data\\{f}')
    data = parse_x_file(filepath)

    # Переведём координаты точек из локальной системы координат в систему координат камеры
    data['vertices'] = apply_transformation(data.get('vertices'), data.get('frame_matrix'))
    
    # Предварительная индексация треугольников
    data["index_map"] = build_triangle_index_map(data, image_size)
    meshes.append(data)

teapot_1.x
num_vertices: 200310
num_faces: 379304
num_uvs: 200310
teapot_2.x
num_vertices: 190307
num_faces: 358169
num_uvs: 190307


2. Инициализируем воксельное пространство

In [158]:
# Сбор всех вершин в единую структуру
all_vertices = np.vstack([mesh["vertices"] for mesh in meshes])

In [159]:
# Границы сцены
min_bound = np.min(all_vertices, axis=0)
max_bound = np.max(all_vertices, axis=0)
scene_size = max_bound - min_bound

In [160]:
# Размер одного вокселя
voxel_size = scene_size / VOXEL_GRID_SIZE

In [161]:
# Инициализация скалярного поля и весов
D = np.zeros((VOXEL_GRID_SIZE, VOXEL_GRID_SIZE, VOXEL_GRID_SIZE), dtype=np.float32)
W = np.zeros_like(D)

In [162]:
# Сохраняем параметры для дальнейших преобразований
voxel_space = {
    "min_bound": min_bound,
    "max_bound": max_bound,
    "scene_size": scene_size,
    "voxel_size": voxel_size,
    "D": D,
    "W": W,
    "grid_size": VOXEL_GRID_SIZE
}

3. Объединяем меши воксельным методом

In [163]:
def compute_triangle_normal(p0, p1, p2):
    return np.cross(p1 - p0, p2 - p0)


def point_to_triangle_distance(p, a, b, c):
    # Истинное расстояние от точки до треугольника в 3D
    ab = b - a
    ac = c - a
    ap = p - a
    
    d1 = np.dot(ab, ap)
    d2 = np.dot(ac, ap)
    if d1 <= 0.0 and d2 <= 0.0:
        # точка p лежит вне треугольника со стороны вершины a
        return np.linalg.norm(ap)

    bp = p - b
    d3 = np.dot(ab, bp)
    d4 = np.dot(ac, bp)
    if d3 >= 0.0 and d4 <= d3:
        # точка p лежит вне треугольника со стороны вершины b
        return np.linalg.norm(bp)

    vc = d1*d4 - d3*d2
    if vc <= 0.0 and d1 >= 0.0 and d3 <= 0.0:
        v = d1 / (d1 - d3)
        proj = a + v * ab
        # точка p ближе всего к ребру AB.
        return np.linalg.norm(p - proj)

    cp = p - c
    d5 = np.dot(ab, cp)
    d6 = np.dot(ac, cp)
    if d6 >= 0.0 and d5 <= d6:
        # точка p лежит вне треугольника со стороны вершины c
        return np.linalg.norm(cp)

    vb = d5*d2 - d1*d6
    if vb <= 0.0 and d2 >= 0.0 and d6 <= 0.0:
        w = d2 / (d2 - d6)
        proj = a + w * ac
        # точка p ближе всего к ребру AC
        return np.linalg.norm(p - proj)

    va = d3*d6 - d5*d4
    if va <= 0.0 and (d4 - d3) >= 0.0 and (d5 - d6) >= 0.0:
        w = (d4 - d3) / ((d4 - d3) + (d5 - d6))
        proj = b + w * (c - b)
        # точка p ближе всего к ребру BC.
        return np.linalg.norm(p - proj)

    # точка p лежит внутри треугольника
    n = np.cross(ab, ac)
    n /= np.linalg.norm(n)
    return abs(np.dot(ap, n))


def compute_weight(p, camera_center, normal):
    direction = camera_center - p
    direction /= np.linalg.norm(direction) + 1e-9
    weight = abs(np.dot(direction, normal))
    return weight


def integrate_mesh_to_voxel_grid(mesh, voxel_space):
    vertices = mesh["vertices"]
    faces = mesh["faces"]
    frame_matrix = mesh["frame_matrix"]
    camera_center = apply_transformation(np.array([[0.0, 0.0, 0.0]]), frame_matrix)[0]

    D = voxel_space["D"]
    W = voxel_space["W"]
    min_bound = voxel_space["min_bound"]
    voxel_size = voxel_space["voxel_size"]
    grid_size = voxel_space["grid_size"]

    for face in tqdm(faces, desc="Faces", unit=" triangles", unit_scale=1):
        a, b, c = vertices[face[0]], vertices[face[1]], vertices[face[2]]
        normal = compute_triangle_normal(a, b, c)
        normal /= np.linalg.norm(normal) + 1e-9

        # Bounding box треугольника в МИРОВЫХ координата
        tri_min = np.minimum(np.minimum(a, b), c)
        tri_max = np.maximum(np.maximum(a, b), c)
        
        # Переводим его в индексы воксельной сетки
        min_idx = np.floor((tri_min - min_bound) / voxel_size).astype(int)
        max_idx = np.ceil((tri_max - min_bound) / voxel_size).astype(int)
        
        # Ограничиваем индексы диапазоном вокселей
        min_idx = np.clip(min_idx, 0, grid_size - 1)
        max_idx = np.clip(max_idx, 0, grid_size - 1)

        for i in range(min_idx[0], max_idx[0] + 1):
            for j in range(min_idx[1], max_idx[1] + 1):
                for k in range(min_idx[2], max_idx[2] + 1):
                    voxel_center = min_bound + voxel_size * (np.array([i, j, k]) + 0.5)
                    distance = point_to_triangle_distance(voxel_center, a, b, c)
                    weight = compute_weight(voxel_center, camera_center, normal)

                    if weight == 0.0:
                        continue  # скипаем неинтересные места

                    D[i, j, k] = (W[i, j, k] * D[i, j, k] + weight * distance) / (W[i, j, k] + weight)
                    W[i, j, k] += weight


In [164]:
# Вычислим интегральную функцию растояния и весовую функцию для каждого вокселя
for mesh in meshes:
    integrate_mesh_to_voxel_grid(mesh, voxel_space)

Faces: 100%|██████████| 379k/379k [03:53<00:00, 1.62k triangles/s] 
Faces: 100%|██████████| 358k/358k [03:46<00:00, 1.58k triangles/s] 


In [78]:
# # Нормализация координат в диапазон [0, GRID_SIZE]
# for mesh in meshes:
#     # Попробуем нормализовать координаты вершан и треугольников под размер воксельной сетки
#     mesh['vertices'] = (mesh.get('vertices') - voxel_space["min_bound"]) / voxel_size

In [66]:
# Если дистанция и вес равны нулю — это значит, что воксель не был затронут треугольниками.
D[W == 0] = np.inf

4. Построение меша

In [97]:
from skimage import measure

In [165]:
safe_D = np.copy(D)

In [151]:
len(safe_D[W == 0])

16460779

In [59]:
# Преобразуем inf в max значение, иначе алгоритм сломается
safe_D[np.isinf(safe_D)] = np.max(safe_D[np.isfinite(safe_D)])

In [42]:
safe_D

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [166]:
# Marching cubes ищет поверхность, где D == 0
verts, faces, normals, values = measure.marching_cubes(safe_D, level=0, spacing=(voxel_space["voxel_size"]))

In [153]:
# # Преобразуем координаты обратно в мировую систему
# verts = verts / VOXEL_GRID_SIZE * voxel_space["scene_size"] + voxel_space["min_bound"]

In [167]:
# Создадим меш на основе вершин и треугольников
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts)
mesh.triangles = o3d.utility.Vector3iVector(faces)

In [168]:
# Пересчитаем нормали
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh.normalize_normals()

TriangleMesh with 275922 points and 551878 triangles.

In [169]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([mesh])

In [47]:
zero_9_D = np.copy(D)

In [104]:
zero_7_D = np.copy(D)

In [122]:
camera_fix_D = np.copy(D)

In [136]:
weight_zero_D = np.copy(D)

In [157]:
abs_D = np.copy(D) # Лучший пока что

In [170]:
abs_only_D = np.copy(D)

In [ ]:
# Переведём координаты в исходный размер
for mesh in meshes:
    mesh['vertices'] = mesh.get('vertices') / VOXEL_GRID_SIZE * voxel_space.get('scene_size') + voxel_space["min_bound"]

In [63]:
len(D[(D == 0) & (W > 0)])

0

In [64]:
len(D[W == 0])

16603396

0

2. Объединяем меши в одну вокселную структуру

In [ ]:
# Чтобы получить больше информации о процессе объединения мешей
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

In [ ]:
# Объединяем меши в один PointCloud
pcd, voxel_size = merge_meshes(meshes, VOXEL_GRID_SIZE, mcd_coarse_scale=60, mcd_fine_scale=12, down_sample=True)

In [ ]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([pcd])

3. Метод Марширующих кубиков

In [ ]:
voxel_grid, origin, scale = pcd_to_voxel_grid(np.asarray(pcd.points), grid_size=VOXEL_GRID_SIZE, apply_filter=False)

In [ ]:
# Построим меш на воксельной структуре
vertices, faces, normals, values = marching_cubes(voxel_grid, voxel_size, level=0.0000001)
# Переведём координаты в исходный размер
vertices = vertices / (VOXEL_GRID_SIZE - 1) * scale + origin

In [ ]:
# Создадим меш на основе вершин и треугольников
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(vertices)
mesh.triangles = o3d.utility.Vector3iVector(faces)

In [ ]:
# Уберём лишнее если есть
mesh.remove_duplicated_vertices()
mesh.remove_duplicated_vertices()
mesh.remove_unreferenced_vertices()
mesh.remove_non_manifold_edges()
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()

In [ ]:
# Пересчитаем нормали
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh.normalize_normals()

In [ ]:
# Посмотрим на результат объединения
o3d.visualization.draw_geometries([mesh])

3.2 Метод Пуассона

In [ ]:
# Построим воксельную структуру из откалиброванных точек всех мешей
voxels, origin, _ = build_voxel_grid(meshes, grid_size=VOXEL_GRID_SIZE)

In [ ]:
# Запустим метод Пуассона
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd=pcd, depth=9)

In [ ]:
# Убрём шум
vertices_to_remove = densities < np.quantile(densities, 0.02)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Пересчитаем нормали
mesh.compute_vertex_normals()
mesh.compute_triangle_normals()
mesh.normalize_normals()

In [ ]:
# Посмотрим на результат
o3d.visualization.draw_geometries([mesh])

3.3 Метод Альфа-формы (Alpha shapes)

In [ ]:
alpha = voxel_size * 0.8
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

4. Сохранение полученного меша

In [171]:
mesh.triangles

std::vector<Eigen::Vector3i> with 551878 elements.
Use numpy.asarray() to access data.

In [172]:
write_mesh_to_x(mesh, 'output_mesh.x', save_normals=False, save_texture=False)

Vertices: 100%|██████████| 276k/276k [00:02<00:00, 130k points/s] 
Faces: 100%|██████████| 552k/552k [00:01<00:00, 307k triangles/s] 
